# An Analysis of Political Contributions During the 2020 House of Representatives Election
Name: Habeeb Kotun Jr.<br>
Team: Silver SLoths<br>
Date: 10/14/2021<br>

In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup as BS
from tqdm.notebook import tqdm

In [2]:
# Scrape the data for all Districts in Tennessee

## Initialize empty dataframe to add data to
TN = pd.DataFrame()

# List of URLs to scrape data from
URLs = ['https://www.opensecrets.org/races/summary?cycle=2020&id=TN01&spec=N', 
        'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N',
        'https://www.opensecrets.org/races/summary?cycle=2020&id=TN03&spec=N', 
        'https://www.opensecrets.org/races/summary?cycle=2020&id=TN04&spec=N',
        'https://www.opensecrets.org/races/summary?cycle=2020&id=TN05&spec=N',
        'https://www.opensecrets.org/races/summary?cycle=2020&id=TN06&spec=N',
        'https://www.opensecrets.org/races/summary?cycle=2020&id=TN07&spec=N',
        'https://www.opensecrets.org/races/summary?cycle=2020&id=TN08&spec=N',
        'https://www.opensecrets.org/races/summary?cycle=2020&id=TN09&spec=N']

# Loop through lists of URLs and extract data from each
for URL in URLs:
    response = requests.get(URL)
    soup = BS(response.text)
    loop_df = pd.read_html(str(soup.find('table')))[0]
    
    # Create state and district columns
    for index in loop_df.index:    
        loop_df.at[index, 'state'] = re.findall(r'id=\w\w\d\d', str(soup.find('link')))[0][3:5]
        loop_df.at[index, 'district'] = re.findall(r'id=\w\w\d\d', str(soup.find('link')))[0][5:7]
    
    # Combine data from webscraping into one dataframe
    if TN.empty:
        TN = loop_df
    else:
        TN = pd.concat([TN, loop_df], ignore_index = True)  
        
TN

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Diana Harshbarger (R) • Winner,"$2,126,946","$1,869,100","$257,846",12/31/2020,TN,01
1,Blair Nicole Walsingham (D),"$140,209","$134,995","$5,215",10/14/2020,TN,01
2,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020,TN,02
3,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020,TN,02
4,Chuck Fleischmann (R) • Incumbent • Winner,"$1,051,653","$381,411","$1,880,341",12/31/2020,TN,03
5,Meg Gorman (D),"$85,843","$77,760","$8,083",12/31/2020,TN,03
6,Scott Desjarlais (R) • Incumbent • Winner,"$331,464","$392,499","$302,649",12/31/2020,TN,04
7,Christopher Hale (D),"$308,731","$302,996","$5,735",12/31/2020,TN,04
8,Jim Cooper (D) • Incumbent • Winner,"$936,569","$1,332,131","$272,934",12/31/2020,TN,05
9,John Rose (R) • Incumbent • Winner,"$1,050,429","$625,688","$454,375",12/31/2020,TN,06


In [3]:
for index in TN.index:
    # Split up string in candidate column
    candidate_split = TN['Candidate'][index].split(' • ')

    # Assign candidate name to candidate column
    TN.at[index, 'Candidate'] = re.sub(r'\s\W\w\W', "", candidate_split[0])
    
    # Create new column named party
    party = re.findall(r'\W\w\W', candidate_split[0])
    TN.at[index, 'party'] = re.sub(r'\W', '', party[0])
    
    # Create column named incumbent and determine incumbent status
    if 'Incumbent' in candidate_split:
        TN.at[index, 'incumbent'] =  True
    else:
        TN.at[index, 'incumbent'] =  False

    # Create column named race and determine if candidate won the race
    if 'Winner' in candidate_split:
        TN.at[index, 'race'] =  True
    else:
        TN.at[index, 'race'] =  False
        
    # Remove non-digit characters from raised and spent columns
    # Also transform raised and spent columns to int type
    TN.at[index, 'Raised'] = int(re.sub(r'\D', '', TN.at[index, 'Raised']))
    TN.at[index, 'Spent'] = int(re.sub(r'\D', '', TN.at[index, 'Spent']))

In [4]:
# Drop unneeded columns
TN = TN.drop(columns=['Cash on Hand', 'Last Report'])

# Rename columns
TN = (TN.rename({'Candidate':'name',
                 'Raised': 'raised',
                 'Spent':'spent'},
                axis='columns'))

TN

,name,raised,spent,state,district,party,incumbent,race
0,Diana Harshbarger,2126946,1869100,TN,01,R,False,True
1,Blair Nicole Walsingham,140209,134995,TN,01,D,False,False
2,Tim Burchett,1336276,878488,TN,02,R,True,True
3,Renee Hoyos,812784,816793,TN,02,D,False,False
4,Chuck Fleischmann,1051653,381411,TN,03,R,True,True
5,Meg Gorman,85843,77760,TN,03,D,False,False
6,Scott Desjarlais,331464,392499,TN,04,R,True,True
7,Christopher Hale,308731,302996,TN,04,D,False,False
8,Jim Cooper,936569,1332131,TN,05,D,True,True
9,John Rose,1050429,625688,TN,06,R,True,True
